In [4]:
import time
import json
from py2neo import Graph, Node, Relationship

In [64]:
graph = Graph("bolt://neo4j-quanta-two:7687", auth=('neo4j','myneo'))
# graph = Graph("bolt://localhost:7687", auth = ('neo4j', 'password'))

print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

Connected to graph database with 15 nodes and 0 relationships!


In [53]:
# Constrain one name per author
print("Creating uniqueness constraint (and also index) on Author names...", end=" ", flush=True)
query = """CREATE CONSTRAINT ON (a:Author2) ASSERT a.name IS UNIQUE;"""
graph.run(query).evaluate()
print("Done.")

Creating uniqueness constraint (and also index) on Author names... Done.


In [ ]:
# Add coauthor relationships on existing graph
query = """
CALL apoc.periodic.iterate(
"MATCH (q:Quanta) WHERE size((q)<-[:AUTHORED]-())>1 RETURN q",
"WITH [(q)<-[:AUTHORED]-(a) | a] as coAuthors
 UNWIND coAuthors as first
 UNWIND coAuthors as second
 WITH first, second
 WHERE id(first) < id(second)
 MERGE (first)-[r:COAUTHOR]-(second)
  ON CREATE SET r.strength = 1
  ON MATCH SET r.strength = r.strength + 1",
{batchSize:10000, iterateList:true, parallel:false});
"""
graph.run(query)

OLD QUERIES

In [ ]:
# Create coauthor relationships - runs on existing graph but double counts
import time

start_time = time.time()

print("Adding coauthor relationships...")
query = """ 
MATCH (q:Quanta)
MATCH (q)<-[:AUTHORED]-(a:Author)
MATCH (q)<-[:AUTHORED]-(b:Author) where a <> b
MERGE (a)-[r:COAUTHOR]-(b)
ON CREATE SET r.strength = 1
ON MATCH SET r.strength = r.strength + 1
"""

query_start_time = time.time()
graph.run(query)#.evaluate()
print("Done in ({:.2f} seconds)...".format((time.time()-query_start_time)))
#print("Done in ({:.2f} minutes)...".format((time.time()-query_start_time)/60))

In [69]:
# Import authors as nodes and create coauthor relationships
import glob, os, time

local_data_dir = '/tmp/data/mag/test' #'/tmp/data/mag/test'
neo4j_data_dir = '/import/mag/test'

start_time = time.time()
for _,_ , files in os.walk(local_data_dir):
    for file in sorted(files):
        if file.endswith('.txt'):
           
            print("Importing {}...".format(file), end=" ", flush=True)
            query = """
            CALL apoc.periodic.iterate(
            "CALL apoc.load.json('file://{}{}') YIELD value AS q RETURN q"
            ,
            "MERGE (p:Quanta {{name:q.title}})
            WITH q, p
            UNWIND q.authors as author 
            MERGE (a:Author2 {{name:author.name}})
            MERGE (a)-[:COCREATOR]->(p)
            WITH p,a
            MATCH (b:Author2)-[]->(p) where a <> b
            MERGE (a)-[r:COAUTHOR]-(b)
            ON CREATE SET r.strength2 = 1
            ON MATCH SET r.strength2 = r.strength2 + 1
            WITH p DETACH DELETE p;"
            , 
            {{batchSize:10000, iterateList:true, parallel:false}});
            """.format(neo4j_data_dir, file)
            
            query_start_time = time.time()
            graph.run(query).evaluate()
            print("Done in ({:.2f} minutes)...".format((time.time()-query_start_time)/60))

print("Finished all imports in {:.2f} hours.".format((time.time()-start_time)/60/60))

Importing nature_author_test.txt... 

ClientError: Procedure Not Found: There is no procedure with the name `apoc.periodic.iterate` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.

In [74]:
query = """
MERGE (p:Quanta2)
WITH q, p
UNWIND q.authors as author 
MERGE (a:Author2 {{name:author.name}})
MERGE (a)-[:COCREATOR]->(p)
WITH p,a
MATCH (b:Author2)-[]->(p) where a <> b
MERGE (a)-[r:COAUTHOR]-(b)
ON CREATE SET r.strength2 = 1
ON MATCH SET r.strength2 = r.strength2 + 1
WITH p DETACH DELETE p
"""

query_start_time = time.time()
graph.run(query).evaluate()
print("Done in ({:.2f} minutes)...".format((time.time()-query_start_time)/60))

print("Finished all imports in {:.2f} hours.".format((time.time()-start_time)/60/60))

CypherSyntaxError: Invalid input '{': expected whitespace, a property key name, '}', an identifier or UnsignedDecimalInteger (line 5, column 19 (offset: 75))
"MERGE (a:Author2 {{name:author.name}})"
                    ^

In [9]:
# Import authors as nodes and create coauthor relationships
import glob, os, time

local_data_dir = '/work/ImportData/CoauthorshipNatureGraph/' #'/tmp/data/mag/test'
neo4j_data_dir = '/work/ImportData/CoauthorshipNatureGraph/'

start_time = time.time()
for _,_ , files in os.walk(local_data_dir):
    for file in sorted(files):
        if file.endswith('.txt'):
           
            print("Importing {}...".format(file), end=" ", flush=True)
            
            query = """CREATE NODE("Author2", name)"""
            
            query = """
            CALL apoc.periodic.iterate(
            "CALL apoc.load.json('file://{}{}') YIELD value AS q RETURN q");
            """.format(neo4j_data_dir, file)
            
            query_start_time = time.time()
            graph.run(query).evaluate()
            print("Done in ({:.2f} minutes)...".format((time.time()-query_start_time)/60))

print("Finished all imports in {:.2f} hours.".format((time.time()-start_time)/60/60))

Finished all imports in 0.00 hours.


In [59]:
def query_to_df(query, graph):
    print("Starting query...", end=" ")
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    return df

In [67]:
query = "MATCH (n:Quanta2) RETURN n"
data = query_to_df(query, graph)
    
data

Starting query... Done (0.00 minutes).


,n
0,"{'title': 'title1', 'venue': 'Nature', 'author..."
1,"{'title': 'title2', 'venue': 'Nature', 'author..."
2,"{'title': 'title3', 'venue': 'Nature', 'author..."
3,"{'title': 'title4', 'venue': 'Nature', 'author..."
4,"{'title': 'title5', 'venue': 'Nature', 'author..."
5,"{'title': 'title6', 'venue': 'PLOS', 'authors'..."
6,"{'title': 'title7', 'venue': 'Cell', 'authors'..."
7,"{'title': 'title8', 'venue': 'Cell', 'authors'..."


In [65]:
d_list = [{"title": "title1", "authors": [{"name": "name1"}], "venue": "Nature"}, {"title": "title2", "authors": [{"name": "name1"}, {"name": "name2"}], "venue": "Nature"}, {"title": "title3", "authors": [{"name": "name1"}, {"name": "name2"}, {"name": "name3"}], "venue": "Nature"}, {"title": "title4", "authors": [{"name": "name2"}, {"name": "name3"}, {"name": "name4"}], "venue": "Nature"}, {"title": "title5", "authors": [{"name": "name5"}, {"name": "name6"}, {"name": "name7"}], "venue": "Nature"}, {"title": "title6", "authors": [{"name": "name1"}, {"name": "name2"}, {"name": "name3"}], "venue": "PLOS"}, {"title": "title7", "authors": [{"name": "name3"}, {"name": "name8"}], "venue": "Cell"}, {"title": "title8", "authors": [{"name": "name4"}, {"name": "name9"}, {"name": "name10"}], "venue": "Cell"}]

for d in d_list:
    node = Node("Quanta2", title=d["title"], authors=[x["name"] for x in d["authors"]], venue=d["venue"])
    graph.create(node)

1
2
3
4
5
6
7
8


In [49]:
graph.delete_all()